### Preliminaries

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
import math
import matplotlib.pyplot as plt
from scipy.ndimage import zoom, rotate
from keras.utils import Sequence
from keras.models import Model
from keras.layers import Dense, Dropout, BatchNormalization, GlobalAveragePooling3D, Activation, Flatten
from keras.layers import MaxPool3D, Conv3D, Input, concatenate, Conv2D, MaxPool2D, GlobalAveragePooling2D
import os
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.decomposition import PCA
import tensorflow as tf
import scipy.io as sio
import glob
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/LIDC/final_data.csv")

In [ ]:
agg_func = {'calcification': 'mean', 'internalStructure': 'mean', 'lobulation': 'mean', 'malignancy': 'mean',
            'margin': 'mean', 'path': 'first', 'sphericity': 'mean', 'spiculation': 'mean', 'subtlety': 'mean', 'texture': 'mean',
            'xf': 'first', 'xi': 'first', 'yf': 'first', 'yi': 'first', 'zf': 'first', 'zi': 'first'}

In [ ]:
df = df.groupby("final_id").agg(agg_func)
df[['calcification', 'internalStructure', 'lobulation', 'malignancy', 'margin', 'sphericity', 'spiculation', 'subtlety', 'texture']] = df[['calcification', 'internalStructure', 'lobulation', 'malignancy', 'margin', 'sphericity', 'spiculation', 'subtlety', 'texture']]/df[['calcification', 'internalStructure', 'lobulation', 'malignancy', 'margin', 'sphericity', 'spiculation', 'subtlety', 'texture']].max()
df = df[['path', 'calcification', 'internalStructure', 'lobulation', 'malignancy',
       'margin', 'sphericity', 'spiculation', 'subtlety', 'texture']]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.index.tolist(), df, stratify=df.malignancy, test_size=0.2, random_state=22)
ids_train = X_train
ids_test = X_test

In [ ]:
tags_train = (df.loc[ids_train].malignancy.values > 0.5).astype(np.int)

In [ ]:
class_weights = class_weight.compute_class_weight('balanced', np.unique(tags_train), tags_train)

In [ ]:
n_classes = 2
chanels = 1
batch_size = 70
vol_size = np.array([42, 42, 42])

path_base = '/content/drive/MyDrive/LIDC/nodules-aug/{}.npz'

### Functions

In [ ]:
def normalize(npzarray):
    maxHU = 400.
    minHU = -1000.
    npzarray = (npzarray - minHU) / (maxHU - minHU)
    npzarray = np.clip(npzarray, 0, 1)
    return npzarray

In [ ]:
def get_data(nodule_id, pos):
    tag = int(df.loc[nodule_id].malignancy > 0.5)
    
    file = np.load(path_base.format(str(nodule_id)+'-'+str(pos)))
    vol = file['patch']
    file.close()
    
    size_vol=np.shape(vol)
    vol = normalize(vol)
    vol = zoom(vol, vol_size/np.array(vol.shape), order=0)
    return vol, tag, size_vol

In [ ]:
#maximum intensity projection
def MIP_data(nodule_id):
  tag = int(df.loc[nodule_id].malignancy > 0.5)
  file = np.load(path_base.format(nodule_id))
  vol = file['patch']
  file.close()

  size_vol=np.shape(vol)
  vol = normalize(vol)
  vol = zoom(vol, vol_size/np.array(vol.shape), order=0)
  vol = np.sum(vol, axis=0)/size_vol[0]

  return vol, tag, size_vol

In [ ]:
def PCA_data(nodule_id):
  tag = int(df.loc[nodule_id].malignancy > 0.5)
  file = np.load(path_base.format(nodule_id))
  vol = file['patch']
  file.close()

  size_vol=np.shape(vol)
  vol = normalize(vol)
  vol = zoom(vol, vol_size/np.array(vol.shape), order=0)
  
  brain_pca = PCA(n_components=3)
  pca=[]
  for zi in range(vol_size[2]):
    pca.append(vol[:,:,zi].flatten())
  brain_pca.fit(pca)
  pca=np.transpose(brain_pca.components_).reshape(vol_size[0],vol_size[1],3)
  for ch in range(3):
    pca[...,ch]=(pca[...,ch]-np.min(pca[...,ch]))/(np.max(pca[...,ch])-np.min(pca[...,ch]))

  return pca, tag, size_vol

In [ ]:
def slices_data(nodule_id):
  tag = int(df.loc[nodule_id].malignancy > 0.5)
  file = np.load(path_base.format(nodule_id))
  vol = file['patch']
  file.close()

  size_vol=np.shape(vol)
  vol = normalize(vol)
  ima=vol[int(size_vol[0]/2)-1:int(size_vol[0]/2)+2]
  ima=tf.image.resize(np.transpose(ima),(vol_size[1], vol_size[2]))

  return ima, tag, size_vol

In [ ]:
degrees = [0, 90, -90, 80, -80, 70, -70]
axis = [0, 1, 2]

def get_random_params():
    d = np.random.choice(degrees)
    a = np.random.choice(axis, size=2, replace=False)
    return d, a

In [ ]:
def plot_vol(vol):
  sv=np.shape(vol)
  ind=np.random.choice(sv[0])
  rows=round(np.sqrt(sv[3]))
  cols=math.ceil(sv[3]/rows)

  plt.figure(figsize=(10,10))
  for j in range(sv[3]):
    plt.subplot(rows,cols,j+1)
    plt.imshow(vol[ind,:,:,j,0])
    plt.title('slice '+str(j))
    plt.axis('off')

In [ ]:
def getitem2(data, batch_size, is_training, idx=0, mode='vol'):

    batch_x = data[idx * batch_size:(idx + 1) * batch_size]
    
    X = []
    y = []
    vol=[]
    
    # se cargan los volumenes y las mascaras en sus respectivos arrays
    for path in batch_x:
        if mode=='vol':
          data, tag, sivol = get_data(path,1)
        if mode=='MIP':
          data, tag, sivol = MIP_data(path)
        if mode=='PCA':
          data, tag, sivol = PCA_data(path)
        if mode=='Slices':
          data, tag, sivol = slices_data(path)
        d, a = get_random_params()
        if is_training:
            data = rotate(data, d, a, reshape=False)
        X.append(data)
        vol.append(sivol)
        
        temp_y = [0, 0]
        temp_y[tag] = 1
        
        y.append(temp_y)
        
    X = np.array(X)
    y = np.array(y)
    vol=np.array(vol)
    
    try:
        X = X.reshape((*X.shape, chanels))
    except Exception as ex:
        print('ojooooooo')
        print(batch_x, ex)
    if mode=='PCA':
      X=X[...,0]
    if mode=='Slices':
      X=X[...,0]
    return X, vol, y

In [ ]:
axix=[(0, 1), (0, 2), (1, 2)]
angle=[0, 90, 180, 270]
flip_ax={"(0, 1)":2,"(0, 2)":1, "(1, 2)":0}
flip_2=[1, 2, 0]

In [ ]:
class Sequence_data(Sequence):

    def __init__(self, data, batch_size, is_training):
        # recive una lista de rutas de donde están los volumenes como numpy arrays y el batch_size
        self.data = data
        self.batch_size = batch_size
        self.is_training = is_training

    def __len__(self):
        return int(np.ceil(len(self.data) / float(self.batch_size)))

    def __getitem__(self, idx):
        # lista de rutas para el batch actual
        batch_x = self.data[idx * self.batch_size:(idx + 1) * self.batch_size]
        
        X = []
        y = []
        vol=[]
        
        # se cargan los volumenes y las mascaras en sus respectivos arrays
        for path in batch_x:
          for j in range(1,25):
            data, tag, vxl = get_data(path,j)

            X.append(data)            
            temp_y = [0, 0]
            temp_y[tag] = 1
            
            y.append(temp_y)
            vol.append(vxl)
            
        X = np.array(X)
        y = np.array(y)
        vol=np.array(vol)
        x0=X
        
        try:
            x0 = X.reshape((*X.shape, chanels))
        except Exception as ex:
            print('ojooooooo')
            print(batch_x, ex)
        return [x0, vol], y

In [ ]:
'''xx=glob.glob('/content/drive/MyDrive/LIDC/nodules-aug/*.npz')
cont=0

for jj in xx:
  file = np.load(jj)
  file.close()
  print('\rProcess ', round(cont*100/64464,4), '%...' , end ="")
  cont=cont+1'''

'xx=glob.glob(\'/content/drive/MyDrive/LIDC/nodules-aug/*.npz\')\ncont=0\n\nfor jj in xx:\n  file = np.load(jj)\n  file.close()\n  print(\'\rProcess \', round(cont*100/64464,4), \'%...\' , end ="")\n  cont=cont+1'

### multipath and multimodal
3D network with input of the nodule dimensions.

In [ ]:
def get_model_MLB(size = (128, 128, 64)):
    
    width, height, depth = size
    inputs = Input((width, height, depth, 1))

    #Layer 1
    l1 = Conv3D(filters=64, kernel_size=3, activation="relu")(inputs)
    l1 = BatchNormalization()(l1)

    l1 = Conv3D(filters=64, kernel_size=3, activation="relu")(l1)
    l1 = BatchNormalization()(l1)

    l1 = Conv3D(filters=128, kernel_size=3, activation="relu")(l1)
    l1 = BatchNormalization()(l1)

    l1 = Conv3D(filters=256, kernel_size=3, activation="relu")(l1)
    l1 = BatchNormalization()(l1)

    l1 = GlobalAveragePooling3D()(l1)
    l1 = Dense(units=150, activation="relu")(l1)
    l1 = Dropout(0.4)(l1)

    #Layer 2
    l2 = Conv3D(filters=64, kernel_size=5, activation="relu")(inputs)
    l2 = BatchNormalization()(l2)

    l2 = Conv3D(filters=64, kernel_size=5, activation="relu")(l2)
    l2 = BatchNormalization()(l2)

    l2 = Conv3D(filters=128, kernel_size=5, activation="relu")(l2)
    l2 = BatchNormalization()(l2)

    l2 = Conv3D(filters=256, kernel_size=5, activation="relu")(l2)
    l2 = BatchNormalization()(l2)

    l2 = GlobalAveragePooling3D()(l2)
    l2 = Dense(units=150, activation="relu")(l2)
    l2 = Dropout(0.3)(l2)

    #Layer 3
    l3 = Conv3D(filters=64, kernel_size=7, activation="relu")(inputs)
    l3 = BatchNormalization()(l3)

    l3 = Conv3D(filters=64, kernel_size=7, activation="relu")(l3)
    l3 = BatchNormalization()(l3)

    l3 = Conv3D(filters=128, kernel_size=7, activation="relu")(l3)
    l3 = BatchNormalization()(l3)

    l3 = Conv3D(filters=256, kernel_size=7, activation="relu")(l3)
    l3 = BatchNormalization()(l3)

    l3 = GlobalAveragePooling3D()(l3)
    l3 = Dense(units=150, activation="relu")(l3)
    l3 = Dropout(0.3)(l3)

    inp2 = Input((3)) #cantidad de datos extras
    
    con = concatenate([l1, l2, l3, inp2])
    
    x = Dense(units=50, activation="relu")(con)
    x = Dropout(0.3)(x)  

    outputs = Dense(units=2, activation="sigmoid")(x)

    # Define the model.
    model = Model(inputs=[inputs, inp2], outputs=[outputs], name="3dcnn")
    return model

In [ ]:
def get_model_2D(size = (42, 42)):
    
    width, height = size

    inputs = Input((width, height, 3))

    x = Conv2D(filters=64, kernel_size=3, activation="relu")(inputs)
    x = MaxPool2D(pool_size=2)(x)
    x = BatchNormalization()(x)
    
    x = Conv2D(filters=128, kernel_size=3, activation="relu")(x)
    x = MaxPool2D(pool_size=2)(x)
    x = BatchNormalization()(x)
    
    x = Conv2D(filters=256, kernel_size=3, activation="relu")(x)
    x = MaxPool2D(pool_size=2)(x)
    x = BatchNormalization()(x)
    
    x = GlobalAveragePooling2D()(x)
    x = Dense(units=200, activation="relu")(x)
    x = Dropout(0.3)(x)

    #inp2 = Input((3)) #cantidad de datos extras
    #x = concatenate([x, inp2])

    x = Dense(units=100, activation="relu")(x)
    x = Dropout(0.3)(x)
    x = Dense(units=50, activation="relu")(x)
    x = Dropout(0.3)(x)   

    outputs = Dense(units=2, activation="sigmoid")(x)

    # Define the model.
    model = Model(inputs=inputs, outputs=outputs, name="2dmip")
    #model = Model(inputs=[inputs, inp2], outputs=[outputs], name="3dcnn")
    return model

In [ ]:
def get_model_3D_multimodal(size = (128, 128, 64)):
    
    width, height, depth = size

    inputs = Input((width, height, depth, 1))

    x = Conv3D(filters=64, kernel_size=3, activation="relu")(inputs)
    x = MaxPool3D(pool_size=2)(x)
    x = BatchNormalization()(x)
    
    x = Conv3D(filters=128, kernel_size=3, activation="relu")(x)
    x = MaxPool3D(pool_size=2)(x)
    x = BatchNormalization()(x)
    
    x = Conv3D(filters=256, kernel_size=3, activation="relu")(x)
    x = MaxPool3D(pool_size=2)(x)
    x = BatchNormalization()(x)
    
    x = GlobalAveragePooling3D()(x)
    x = Dense(units=200, activation="relu")(x)
    x = Dropout(0.3)(x)

    inp2 = Input((3)) #cantidad de datos extras
    con = concatenate([x, inp2])

    x = Dense(units=100, activation="relu")(con)
    x = Dropout(0.3)(x)
    x = Dense(units=50, activation="relu")(x)
    x = Dropout(0.3)(x)   

    outputs = Dense(units=2, activation="sigmoid")(x)

    # Define the model.
    model = Model(inputs=[inputs, inp2], outputs=[outputs], name="3dcnn")
    return model

### Data

In [ ]:
ids=np.sort(np.concatenate((X_train,X_test)))
sis=np.shape(ids)[0]
volumes=[]

for nu, id in enumerate(ids):
  _,_,siz=get_data(id, 1)
  volumes.append(siz)
  print('\rProcess ', round(nu*100/sis,4), '%...' , end ="")
volumes=np.array(volumes)

Process  99.9628 %...

In [ ]:
vol_m=volumes[...,0]*volumes[...,1]*volumes[...,2]
ids_v=np.argsort(vol_m)

### Model

In [ ]:
path_s='/content/drive/MyDrive/Indigo/weights_cancerPruebaAug'

In [ ]:
if os.path.exists(path_s):
  print('done')
else:
  os.system('mkdir '+path_s)

In [ ]:
model = get_model_3D_multimodal((42,42,42))
model.save(path_s+'/cancer_model.hdf5')
model.summary()

Model: "3dcnn"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 42, 42, 42,  0                                            
__________________________________________________________________________________________________
conv3d (Conv3D)                 (None, 40, 40, 40, 6 1792        input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling3d (MaxPooling3D)    (None, 20, 20, 20, 6 0           conv3d[0][0]                     
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 20, 20, 20, 6 256         max_pooling3d[0][0]              
______________________________________________________________________________________________

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='Adadelta', metrics=['accuracy'])

In [ ]:
weight = {i : class_weights[i] for i in range(2)}

In [ ]:
split=4
ind_s=int(np.shape(ids)[0]/split)

In [ ]:
def indices(n_split):
  data_s=ids[ids_v][ind_s*n_split:ind_s*(n_split+1)]
  ram=np.random.choice(ind_s, ind_s, replace=False)
  data_s = data_s[ram]

  trai = data_s[:int(ind_s*0.7)]
  vali = data_s[int(ind_s*0.7):int(ind_s*0.8)]
  test = data_s[int(ind_s*0.8):]
  return trai, vali, test

In [ ]:
batch_size=3
name_n="/Aug_3D_multimodal_split_"
for j in range(split):
    
  ids_train, ids_val, ids_test=indices(j)

  seq_train = Sequence_data(ids_train, batch_size, True)
  seq_valid = Sequence_data(ids_val, batch_size, False)

  ES = EarlyStopping(patience=20, min_delta=0.00001, restore_best_weights=True)
  MCP = ModelCheckpoint(filepath=path_s+name_n+str(j)+"_.{epoch:02d}.hdf5", save_best_only=False, save_weights_only=True)
  callbacks = [ES, MCP]

  results = model.fit(seq_train, validation_data=(seq_valid),  class_weight=weight , steps_per_epoch=int(len(ids_train)/batch_size), epochs=200, callbacks=callbacks)
  sio.savemat(path_s+name_n+str(j)+'_r.mat', results.history)

Epoch 1/200
156/156 [==============================] - 2930s 19s/step - loss: 0.9884 - accuracy: 0.4611 - val_loss: 0.8229 - val_accuracy: 0.3731
Epoch 2/200
156/156 [==============================] - 45s 286ms/step - loss: 0.9164 - accuracy: 0.5029 - val_loss: 0.8171 - val_accuracy: 0.3731
Epoch 3/200
156/156 [==============================] - 45s 286ms/step - loss: 0.8774 - accuracy: 0.5198 - val_loss: 0.7621 - val_accuracy: 0.4366
Epoch 4/200
156/156 [==============================] - 45s 284ms/step - loss: 0.8564 - accuracy: 0.5243 - val_loss: 0.6785 - val_accuracy: 0.5721
Epoch 5/200
156/156 [==============================] - 44s 283ms/step - loss: 0.8393 - accuracy: 0.5316 - val_loss: 0.6352 - val_accuracy: 0.5752
Epoch 6/200
156/156 [==============================] - 45s 289ms/step - loss: 0.8266 - accuracy: 0.5419 - val_loss: 0.6229 - val_accuracy: 0.5933
Epoch 7/200
156/156 [==============================] - 44s 282ms/step - loss: 0.8107 - accuracy: 0.5570 - val_loss: 0.6181 -

### Graphics

In [ ]:
def plot_t(his_m, mtc='accuracy', name='none'):
  plt.figure(figsize=(15,5))
  plt.subplot(1,2,1)
  plt.plot(his_m[mtc][0])
  plt.plot(his_m['val_'+mtc][0])
  plt.title(mtc)
  plt.ylabel(mtc)
  plt.ylim((0, 1))
  plt.xlabel('epoch')
  plt.legend(['train', 'test'], loc='upper left')

  plt.subplot(1,2,2)
  plt.plot(his_m['loss'][0])
  plt.plot(his_m['val_loss'][0])
  plt.title('model loss')
  plt.ylabel('value')
  plt.ylim((0, 1))
  plt.xlabel('epoch')
  plt.legend(['train', 'test'], loc='upper left')

  if not name=='none':
    plt.savefig(name)

In [ ]:
histories=glob.glob(path_s+'/*.mat')

In [ ]:
plot_t(sio.loadmat(histories[3]))

### Test data

In [ ]:
x1_test, x2_test, y_test=getitem2(ids_test, np.shape(ids_test)[0], False)

In [ ]:
results = model.evaluate([x1_test, x2_test], y_test)

y_hat=model.predict([x1_test, x2_test])

5/5 [==============================] - 0s 40ms/step - loss: 0.3503 - accuracy: 0.8741


In [ ]:
print(classification_report(np.argmax(y_test, axis=1), np.argmax(y_hat, axis=1), digits=5))

              precision    recall  f1-score   support

           0    0.72727   0.36364   0.48485        22
           1    0.88710   0.97345   0.92827       113

    accuracy                        0.87407       135
   macro avg    0.80718   0.66854   0.70656       135
weighted avg    0.86105   0.87407   0.85601       135

